In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle

from gpu import set_gpu
import os
import copy as cp
from sklearn.utils import shuffle

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from DL_utils import MLP, MLPbeta, CNN

import process_data as prd
set_gpu()

Num GPUs Available:  1


In [2]:
sub_type = 'TR'
with open('train_data_raw_'  + sub_type + '.p', 'rb') as f:
    raw, params,feat,feat_sq = pickle.load(f)

In [3]:
## Training 
sub = 1
train_grp = 2
n_train = 'fullallmix4'
train_scale = 5
cv_type = 'manual'
scaler_load = False
feat_type = 'feat'

ind = (params[:,0] == sub) & (params[:,3] == train_grp)

x_train, x_test, x_valid, p_train, p_test, p_valid = prd.train_data_split(raw,params,sub,sub_type,dt=cv_type,load=True,train_grp=train_grp)

emg_scale = np.ones((np.size(x_train,1),1))
for i in range(np.size(x_train,1)):
    emg_scale[i] = 5/np.max(np.abs(x_train[:,i,:]))
x_train = x_train*emg_scale
x_valid = x_valid*emg_scale

x_train_noise, x_train_clean, y_train_clean = prd.add_noise(x_train, p_train, sub, n_train, train_scale)
x_valid_noise, x_valid_clean, y_valid_clean = prd.add_noise(x_train, p_train, sub, n_train, train_scale)

# shuffle data to make even batches
x_train_noise, x_train_clean, y_train_clean = shuffle(x_train_noise, x_train_clean, y_train_clean, random_state = 0)

# Extract features
scaler = MinMaxScaler(feature_range=(0,1))
x_train_noise_cnn, scaler = prd.extract_scale(x_train_noise,scaler,scaler_load,ft=feat_type,emg_scale=emg_scale) 
x_train_clean_cnn, _ = prd.extract_scale(x_train_clean,scaler,ft=feat_type,emg_scale=emg_scale)
x_valid_noise_cnn, _ = prd.extract_scale(x_valid_noise,scaler,ft=feat_type,emg_scale=emg_scale)
x_valid_clean_cnn, _ = prd.extract_scale(x_valid_clean,scaler,ft=feat_type,emg_scale=emg_scale)

# reshape data for nonconvolutional network
x_train_noise_mlp = x_train_noise_cnn.reshape(x_train_noise_cnn.shape[0],-1)
x_valid_noise_mlp = x_valid_noise_cnn.reshape(x_valid_noise_cnn.shape[0],-1)

# create batches
trainmlp_ds = tf.data.Dataset.from_tensor_slices((x_train_noise_mlp.astype('float32'), y_train_clean)).batch(128)
testmlp_ds = tf.data.Dataset.from_tensor_slices((x_valid_noise_mlp.astype('float32'), y_valid_clean)).batch(128)
traincnn_ds = tf.data.Dataset.from_tensor_slices((x_train_noise_cnn.astype('float32'), y_train_clean)).batch(128)
testcnn_ds = tf.data.Dataset.from_tensor_slices((x_valid_noise_cnn.astype('float32'), y_valid_clean)).batch(128)

Loading training data: traindata_manual/TR1_traindata_2.p


In [4]:
mlp = MLP()
mlp_beta = MLPbeta()
cnn = CNN()

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

## TRAIN TEST MLP
@tf.function
def train_mlp(x, y):
    with tf.GradientTape() as tape:
        y_out = mlp(x)
        loss = loss_fn(y, y_out)
    gradients = tape.gradient(loss, mlp.trainable_variables)
    optimizer.apply_gradients(zip(gradients, mlp.trainable_variables))

    train_loss(loss)
    train_accuracy(y, y_out)

@tf.function
def test_mlp(x, y):
    y_out = mlp(x)
    t_loss = loss_fn(y, y_out)

    test_loss(t_loss)
    test_accuracy(y, y_out)

## TRAIN TEST MLP BETA
@tf.function
def train_mlpbeta(x, y):
    with tf.GradientTape() as tape:
        y_out = mlp_beta(x)
        loss = loss_fn(y, y_out)
    gradients = tape.gradient(loss, mlp_beta.trainable_variables)
    optimizer.apply_gradients(zip(gradients, mlp_beta.trainable_variables))

    train_loss(loss)
    train_accuracy(y, y_out)

@tf.function
def test_mlpbeta(x, y):
    y_out = mlp_beta(x)
    t_loss = loss_fn(y, y_out)

    test_loss(t_loss)
    test_accuracy(y, y_out)

## TRAIN TEST CNN
@tf.function
def train_cnn(x, y):
    with tf.GradientTape() as tape:
        y_out = cnn(x)
        loss = loss_fn(y, y_out)
    gradients = tape.gradient(loss, cnn.trainable_variables)
    optimizer.apply_gradients(zip(gradients, cnn.trainable_variables))

    train_loss(loss)
    train_accuracy(y, y_out)

@tf.function
def test_cnn(x, y):
    y_out = cnn(x)
    t_loss = loss_fn(y, y_out)

    test_loss(t_loss)
    test_accuracy(y, y_out)

In [6]:
EPOCHS = 10
models = ['mlp','mlpbeta','cnn']

for model in models:
    print('Training ' + model)
    for epoch in range(EPOCHS):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

        # Train MLP
        if model == 'mlp':
            for x, y in trainmlp_ds:
                train_mlp(x, y)
            for x_test, y_test in testmlp_ds:
                test_mlp(x_test, y_test)
        # Train MLP Beta
        elif model == 'mlpbeta':
            for x, y in trainmlp_ds:
                train_mlpbeta(x, y)
            for x_test, y_test in testmlp_ds:
                test_mlpbeta(x_test, y_test)
        # Train CNN
        elif model == 'cnn':
            for x, y in traincnn_ds:
                train_cnn(x,y)
            for x_test, y_test in testcnn_ds:
                test_cnn(x_test, y_test)

        print(
            f'Epoch {epoch + 1}, '
            f'Loss: {train_loss.result()}, '
            f'Accuracy: {train_accuracy.result() * 100}, '
            f'Test Loss: {test_loss.result()}, '
            f'Test Accuracy: {test_accuracy.result() * 100}'
        )

Training mlp
Epoch 1, Loss: 1.9270107746124268, Accuracy: 21.559524536132812, Test Loss: 1.8660930395126343, Test Accuracy: 25.349205017089844
Epoch 2, Loss: 1.811282992362976, Accuracy: 35.599205017089844, Test Loss: 1.7665140628814697, Test Accuracy: 39.79364776611328
Epoch 3, Loss: 1.7478357553482056, Accuracy: 41.39285659790039, Test Loss: 1.7335928678512573, Test Accuracy: 43.32142639160156
Epoch 4, Loss: 1.727824091911316, Accuracy: 43.30158996582031, Test Loss: 1.7210471630096436, Test Accuracy: 44.55158615112305
Epoch 5, Loss: 1.7170411348342896, Accuracy: 44.369049072265625, Test Loss: 1.711540937423706, Test Accuracy: 45.166664123535156
Epoch 6, Loss: 1.7091256380081177, Accuracy: 45.30555725097656, Test Loss: 1.7051315307617188, Test Accuracy: 45.53571319580078
Epoch 7, Loss: 1.702701449394226, Accuracy: 45.80952453613281, Test Loss: 1.6993718147277832, Test Accuracy: 46.07936477661133
Epoch 8, Loss: 1.6974685192108154, Accuracy: 46.33333206176758, Test Loss: 1.6943593025207

KeyboardInterrupt: 